In [1]:
import threading
import queue

import time
import re
import os
from os import path
from tika import parser

import spacy
import bs4 as bs
import json

from tqdm import tqdm

DIRETORIO_PDF = 'BauruPDF/'
DIRETORIO_TXT = 'BauruTXT/'
DIRETORIO_DOC = 'BauruDOC/'
DIRETORIO_PAR = 'BauruPAR/'

nlp_exclude_list = ['parser',
                    'ner',
                    'entity_linker',
                    'entity_ruler',
                    'textcat',
                    'textcat_multilabel',
                    'lemmatizer',
                    'trainable_lemmatizer',
                    'morphologizer',
                    'attribute_ruler',
                    'senter',
                    'sentencizer',
                    'tok2vec',
                    'transformer']
global_nlp = spacy.load('pt_core_news_lg', exclude = nlp_exclude_list)

In [3]:
dos_list = os.listdir(DIRETORIO_PAR)
dos_len = len(dos_list)

do_current = 0

do_queue = queue.Queue()
for do in dos_list:
    do_queue.put(do)

n_threads = 2

thread_queue = queue.Queue(maxsize = n_threads)

def producer():
    while not do_queue.empty():
        if not thread_queue.full():
            thread_queue.put(do_queue.get())
            do_queue.task_done()
        else:
            time.sleep(0.1)
            
    return

def consumer(name):
    local = threading.local()
    local.name = name
    local.nlp = spacy.load('pt_core_news_lg', exclude = nlp_exclude_list)
    while not do_queue.empty() or not thread_queue.empty():
        if not thread_queue.empty():
            do = thread_queue.get()
            local.time_in = time.time()
            analisa_paragrafos(do, do + '.json_spacy', nlp = local.nlp)
            local.time_out = time.time()
            local.now = time.strftime('%H:%M:%S')
            print(f'{local.name} {local.now} ({(local.time_out - local.time_in):4.5f}): {do}')
            thread_queue.task_done()
        else:
            time.sleep(0.1)
    return
            
producer_thread = threading.Thread(target = producer, name = 'produtor')
producer_thread.start()

print('sleeping for 2 seconds')
time.sleep(2)

consumer_threads = []
consumer_nlps = []
for c in range(n_threads):
    consumer_thread = threading.Thread(target = consumer, args = (str(c)), name = 'consumidor' + str(c))
    consumer_threads.append(consumer_thread)
    
for c in consumer_threads:
    c.start()
    
for c in consumer_threads:
    c.join()

print('done??')

sleeping for 2 seconds
1 20:20:32 (4.83895): do_20180116_2923.pdf.xhtml.json
0 20:20:34 (6.59224): do_20160324_2660.pdf.xhtml.json
0 20:20:39 (5.34589): do_20200204_3221.pdf.xhtml.json
1 20:20:39 (7.29612): do_20170701_2846.pdf.xhtml.json
0 20:20:45 (5.71323): do_20200328_3243.pdf.xhtml.json
0 20:20:49 (4.04886): do_20190117_3067.pdf.xhtml.json
1 20:20:51 (11.24564): do_20171216_2912.pdf.xhtml.json
0 20:20:51 (1.96219): do_20170321_2806.pdf.xhtml.json
0 20:20:56 (4.91139): do_20181229_3060.pdf.xhtml.json
0 20:20:59 (2.87383): do_20170525_2831.pdf.xhtml.json
1 20:20:59 (8.54295): do_20200704_3280.pdf.xhtml.json
1 20:21:00 (0.78226): do_20170207_2789.pdf.xhtml.json
0 20:21:02 (3.61045): do_20160514_2680.pdf.xhtml.json
0 20:21:08 (5.28365): do_20160409_2666.pdf.xhtml.json
1 20:21:08 (8.01541): do_20190202_3074.pdf.xhtml.json
1 20:21:14 (6.37718): do_20170826_2869.pdf.xhtml.json
1 20:21:18 (3.11215): do_20200123_3216.pdf.xhtml.json
1 20:21:21 (3.49564): do_20190514_3114.pdf.xhtml.json
0 20

0 20:25:37 (3.42994): do_20171007_2886.pdf.xhtml.json
0 20:25:43 (6.57415): do_20220120_3504.pdf.xhtml.json
1 20:25:44 (9.54239): do_20170930_2883.pdf.xhtml.json
0 20:25:46 (2.59541): do_20180421_2962.pdf.xhtml.json
1 20:25:53 (9.54019): do_20180424_2963.pdf.xhtml.json
0 20:25:53 (7.58854): do_20170916_2877.pdf.xhtml.json
0 20:25:56 (2.71227): do_20190725_3144.pdf.xhtml.json
1 20:25:56 (2.71245): do_20180313_2946.pdf.xhtml.json
1 20:25:58 (1.78235): do_20160719_2707.pdf.xhtml.json
0 20:26:01 (4.93577): do_20201217_3346.pdf.xhtml.json
0 20:26:04 (2.63780): do_20190427_3108.pdf.xhtml.json
1 20:26:05 (7.08721): do_20191130_3195.pdf.xhtml.json
1 20:26:07 (2.12100): do_20171205_2907.pdf.xhtml.json
0 20:26:07 (3.52059): do_20160521_2683.pdf.xhtml.json
1 20:26:09 (1.63012): do_20160128_2637.pdf.xhtml.json
1 20:26:11 (2.65925): do_20211118_3479.pdf.xhtml.json
1 20:26:15 (3.40384): do_20161117_2754.pdf.xhtml.json
0 20:26:15 (8.06029): do_20220319_3528.pdf.xhtml.json
0 20:26:17 (2.13009): do_201

1 20:29:47 (3.42366): do_20180118_2924.pdf.xhtml.json
0 20:29:48 (2.04662): do_20200604_3268.pdf.xhtml.json
1 20:29:49 (1.22662): do_20210518_3407.pdf.xhtml.json
0 20:29:50 (2.39800): do_20190323_3094.pdf.xhtml.json
0 20:29:52 (1.92617): do_20180120_2925.pdf.xhtml.json
1 20:29:53 (4.22846): do_20201105_3329.pdf.xhtml.json
1 20:29:55 (1.88458): do_20210209_3367.pdf.xhtml.json
0 20:29:57 (5.13324): do_20160223_2647.pdf.xhtml.json
0 20:29:59 (2.21722): do_20191126_3193.pdf.xhtml.json
1 20:30:02 (6.86684): do_20181122_3045.pdf.xhtml.json
0 20:30:03 (3.29071): do_20201112_3332.pdf.xhtml.json
1 20:30:07 (5.14748): do_20160702_2700.pdf.xhtml.json
0 20:30:08 (5.31592): do_20210911_3454.pdf.xhtml.json
1 20:30:08 (1.54644): do_20200523_3264.pdf.xhtml.json
0 20:30:11 (2.69047): do_20170629_2845.pdf.xhtml.json
1 20:30:11 (2.73514): do_20200929_3316.pdf.xhtml.json
0 20:30:13 (1.90124): do_20160917_2731.pdf.xhtml.json
1 20:30:13 (2.32600): do_20181106_3040.pdf.xhtml.json
0 20:30:15 (1.99817): do_202

0 20:34:02 (4.36833): do_20200926_3315.pdf.xhtml.json
0 20:34:04 (1.62068): do_20200519_3262.pdf.xhtml.json
0 20:34:05 (1.83108): do_20190713_3139.pdf.xhtml.json
1 20:34:06 (4.83085): do_20160820_2720.pdf.xhtml.json
0 20:34:08 (2.32737): do_20210610_3416.pdf.xhtml.json
0 20:34:10 (2.26207): do_20180306_2943.pdf.xhtml.json
1 20:34:10 (4.79893): do_20201114_3333.pdf.xhtml.json
1 20:34:12 (1.27635): do_20190302_3086.pdf.xhtml.json
1 20:34:13 (1.62457): do_20170307_2800.pdf.xhtml.json
0 20:34:14 (4.00352): do_20201119_3335.pdf.xhtml.json
0 20:34:16 (1.79648): do_20170921_2879.pdf.xhtml.json
1 20:34:16 (2.71575): do_20191012_3177.pdf.xhtml.json
1 20:34:17 (1.04692): do_20210427_3398.pdf.xhtml.json
0 20:34:19 (3.64301): do_20190521_3117.pdf.xhtml.json
1 20:34:20 (2.56597): do_20211125_3482.pdf.xhtml.json
1 20:34:23 (2.94877): do_20200121_3215.pdf.xhtml.json
0 20:34:23 (3.58788): do_20210717_3431.pdf.xhtml.json
1 20:34:25 (2.45440): do_20200827_3303.pdf.xhtml.json
1 20:34:26 (1.28632): do_202

1 20:38:04 (4.26978): do_20181018_3033.pdf.xhtml.json
1 20:38:08 (4.10014): do_20190430_3109.pdf.xhtml.json
1 20:38:11 (3.01043): do_20170420_2818.pdf.xhtml.json
1 20:38:13 (1.53359): do_20210720_3432.pdf.xhtml.json
1 20:38:14 (0.75439): do_20200407_3247.pdf.xhtml.json
1 20:38:18 (3.79693): do_20220517_3551.pdf.xhtml.json
1 20:38:20 (2.43679): do_20220115_3502.pdf.xhtml.json
0 20:38:21 (18.41369): do_20220519_3552.pdf.xhtml.json
1 20:38:23 (2.55743): do_20170506_2823.pdf.xhtml.json
0 20:38:23 (2.92070): do_20220210_3513.pdf.xhtml.json
0 20:38:25 (2.01016): do_20190829_3158.pdf.xhtml.json
0 20:38:26 (1.01011): do_20220127_3507.pdf.xhtml.json
0 20:38:32 (5.74611): do_20170810_2862.pdf.xhtml.json
1 20:38:33 (10.11371): do_20190105_3062.pdf.xhtml.json
0 20:38:34 (1.81049): do_20220222_3518.pdf.xhtml.json
1 20:38:37 (4.82998): do_20220310_3524.pdf.xhtml.json
1 20:38:40 (2.67928): do_20210629_3424.pdf.xhtml.json
0 20:38:42 (7.68758): do_20160426_2672.pdf.xhtml.json
1 20:38:45 (4.69654): do_2

0 20:42:31 (4.12598): do_20170114_2779.pdf.xhtml.json
0 20:42:35 (3.68228): do_20161126_2758.pdf.xhtml.json
1 20:42:36 (27.18370): do_20171230_2917.pdf.xhtml.json
0 20:42:38 (3.04744): do_20170819_2866.pdf.xhtml.json
1 20:42:39 (3.73900): do_20170511_2825.pdf.xhtml.json
1 20:42:41 (1.79807): do_20200630_3278.pdf.xhtml.json
0 20:42:41 (3.73107): do_20181023_3035.pdf.xhtml.json
0 20:42:45 (3.11868): do_20160505_2676.pdf.xhtml.json
1 20:42:47 (5.65825): do_20201224_3349.pdf.xhtml.json
0 20:42:48 (3.45805): do_20181113_3043.pdf.xhtml.json
0 20:42:50 (1.71757): do_20200903_3306.pdf.xhtml.json
0 20:42:52 (2.27815): do_20160604_2688.pdf.xhtml.json
1 20:42:54 (6.82950): do_20180913_3019.pdf.xhtml.json
0 20:42:54 (2.13764): do_20180324_2951.pdf.xhtml.json
0 20:42:55 (0.92581): do_20161213_2765.pdf.xhtml.json
1 20:42:55 (1.48586): do_20171107_2897.pdf.xhtml.json
0 20:42:58 (2.38966): do_20200922_3313.pdf.xhtml.json
1 20:42:58 (2.77007): do_20171026_2893.pdf.xhtml.json
0 20:42:59 (1.21911): do_20

0 20:46:27 (4.58042): do_20210429_3399.pdf.xhtml.json
1 20:46:30 (6.29574): do_20160213_2643.pdf.xhtml.json
1 20:46:34 (4.60733): do_20210619_3420.pdf.xhtml.json
0 20:46:34 (7.57998): do_20190413_3103.pdf.xhtml.json
1 20:46:38 (4.03009): do_20160512_2679.pdf.xhtml.json
0 20:46:39 (4.44605): do_20190423_3106.pdf.xhtml.json
1 20:46:40 (2.20271): do_20190115_3066.pdf.xhtml.json
0 20:46:41 (1.84211): do_20201222_3348.pdf.xhtml.json
0 20:46:43 (2.64049): do_20220402_3534.pdf.xhtml.json
0 20:46:44 (1.07598): do_20200825_3302.pdf.xhtml.json
1 20:46:51 (10.05870): do_20210109_3354.pdf.xhtml.json
0 20:46:51 (6.50820): do_20171130_2905.pdf.xhtml.json
0 20:46:53 (1.76192): do_20190921_3168.pdf.xhtml.json
1 20:46:53 (2.80976): do_20220526_3555.pdf.xhtml.json
1 20:46:58 (4.51425): do_20181110_3042.pdf.xhtml.json
0 20:46:59 (6.56217): do_20191005_3174.pdf.xhtml.json
0 20:47:02 (2.25593): do_20211026_3472.pdf.xhtml.json
0 20:47:04 (1.91280): do_20171125_2903.pdf.xhtml.json
1 20:47:07 (8.93305): do_20

In [2]:

def raspa_pdf(origem_filename, destino_filename, origem_diretorio = DIRETORIO_PDF, destino_diretorio = DIRETORIO_TXT):
    """
    Gera um arquivo .txt a partir dos dados de um arquivo .pdf utilizando um servidor Apache Tika.
    
    :param origem_filename str: Nome do arquivo de entrada em .pdf
    :param destino_filename str: Nome do arquivo de saída em .xhtml
    :keyword origem_diretorio str: Caminho do diretório do arquivo de entrada
    :keyword destino_diretorio str: Caminho do diretório do arquivo de saída
    
    :returns: `destino_filename'
    """
    
    assert path.splitext(origem_filename)[1].lower()  == '.pdf',             f'Arquivo origem "{origem_filename}" não é .pdf'
    assert path.isfile(path.join(origem_diretorio, origem_filename)),        f'Arquivo origem "{origem_filename}" não existe'
    assert path.splitext(destino_filename)[1].lower() == '.xhtml',           f'Arquivo destino "{destino_filename}" não é .xhtml'
    assert not path.isfile(path.join(destino_diretorio, destino_filename)),  f'Arquivo destino "{destino_filename}" já existe'
    
    conteudo = parser.from_file(path.join(origem_diretorio, origem_filename), xmlContent = True)
    
    with open(path.join(destino_diretorio, destino_filename), 'w') as arquivo_destino:
        arquivo_destino.write(str(conteudo['content']))
    return destino_filename
        

def nome_arquivo_edicao(edicao, diretorio = DIRETORIO_PDF):
    """
    Determina o nome do arquivo .pdf da n-ésima Edição do Diário Oficial de Bauru.
    
    :param edicao int: número da Edição do Diário Oficial de Bauru.
    :keyword diretorio str: Caminho do diretório a buscar o arquivo .pdf.
    
    :returns: o nome do arquivo .pdf ou None, se não encontrado
    """
    
    assert type(edicao) == int, 'Argumento \'edicao\' não é int'
    
    for nome_arquivo in os.listdir(diretorio):
        if re.match('do_[0-9]{8}_' + str(edicao) + '.pdf', nome_arquivo):
            return nome_arquivo
        
    return None

def limpa_txt(origem_filename, destino_filename, origem_diretorio = DIRETORIO_TXT, destino_diretorio = DIRETORIO_PAR):
    """
    Utiliza informações levantadas durante a busca exploratória para extrair apenas os dados relevantes do Diário Oficial de Bauru.
    Ou seja, remove os dados não relevantes como metadados e cabeçalhos de página e separa por parágrafos em uma lista JSON.
    
    :param origem_filename str: Nome do arquivo de entrada em .xhtml
    :param destino_filename str: Nome do arquivo de saída em .json
    :keyword origem_diretorio str: Caminho do diretório do arquivo de entrada
    :keyword destino_diretorio str: Caminho do diretório do arquivo de saída
    
    :returns: nome do arquivo com a lista de parágrafos
    """
    
    assert path.splitext(origem_filename)[1].lower()  == '.xhtml',           f'Arquivo origem "{origem_filename}" não é .xhtml'
    assert path.isfile(path.join(origem_diretorio, origem_filename)),        f'Arquivo origem "{origem_filename}" não existe'
    assert path.splitext(destino_filename)[1].lower() == '.json',            f'Arquivo destino "{destino_filename}" não é .json'
    assert not path.isfile(path.join(destino_diretorio, destino_filename)),  f'Arquivo destino "{destino_filename}" já existe'
    
    with open(path.join(origem_diretorio, origem_filename), 'rb') as arquivo:
        soup = bs.BeautifulSoup(arquivo, 'html.parser')
    
    # Remove cabeçalho do arquivo composto de tags meta
    #   Não faz parte do conteúdo do documento
    soup.head.decompose()
    
    # Planifica as páginas (em divs) do documento
    for div in soup.find_all('div'):
        div.unwrap()    
        
    # Remove os cabeçalhos das páginas
    re_comp_pagina = re.compile('\d+\s?diário oficial de bauru', flags = re.IGNORECASE)
    for paragrafo in soup.find_all('p'):
        re_pagina = re_comp_pagina.match(paragrafo.text)
        if re_pagina != None:
            paragrafo.decompose()
            
    # Analisa cada parágrafo do arquivo com o Spacy e cria um único Doc do Diário Oficial de Bauru inteiro
    lista_docs = []
    for paragrafo in soup.find_all('p'):
        lista_docs.append(paragrafo.text)
        
    with open(path.join(destino_diretorio, destino_filename), 'w') as arquivo:
        json.dump(lista_docs, arquivo, ensure_ascii = False)
        
    return destino_filename
    
def analisa_paragrafos(origem_filename, destino_filename, origem_diretorio = DIRETORIO_PAR, destino_diretorio = DIRETORIO_DOC, nlp = global_nlp):
    """
    Passa os parágrafos pelo pipeline do Spacy e salva os Docs gerados em uma lista JSON.
    
    :param origem_filename str: Nome do arquivo de entrada em .json
    :param destino_filename str: Nome do arquivo de saída em .json_spacy
    :keyword origem_diretorio str: Caminho do diretório do arquivo de entrada
    :keyword destino_diretorio str: Caminho do diretório do arquivo de saída
    
    :returns: nome do arquivo com a lista de parágrafos
    """
    
    assert path.splitext(origem_filename)[1].lower()  == '.json',            f'Arquivo origem "{origem_filename}" não é .json'
    assert path.isfile(path.join(origem_diretorio, origem_filename)),        f'Arquivo origem "{origem_filename}" não existe'
    assert path.splitext(destino_filename)[1].lower() == '.json_spacy',      f'Arquivo destino "{destino_filename}" não é .json_spacy'
    assert not path.isfile(path.join(destino_diretorio, destino_filename)),  f'Arquivo destino "{destino_filename}" já existe'
    
    with open(path.join(origem_diretorio, origem_filename), 'r') as arquivo:
        lista_paragrafos = json.load(arquivo)
        
    lista_docs = []
    for paragrafo in lista_paragrafos:
        lista_docs.append(nlp(paragrafo).to_json())
    
    with open(path.join(destino_diretorio, destino_filename), 'w') as arquivo:
        json.dump(lista_docs, arquivo, ensure_ascii = False)
    
    #doc = spacy.tokens.Doc.from_docs(lista_docs)
    #doc.to_disk(path.join(destino_diretorio, destino_filename))
    
    return destino_filename

In [3]:
verifica_diario_rotineiro = re.compile('do_\d{8}_\d{4}.pdf')
for arquivo_pdf in tqdm(os.listdir(DIRETORIO_PDF)):
    if verifica_diario_rotineiro.match(arquivo_pdf):
        raspa_pdf(arquivo_pdf, arquivo_pdf + '.xhtml')

for arquivo_xhtml in tqdm(os.listdir(DIRETORIO_TXT)):
    limpa_txt(arquivo_xhtml, arquivo_xhtml + '.json')

100%|██████████| 950/950 [15:29<00:00,  1.02it/s] 


In [ ]:
edicao = nome_arquivo_edicao(3546)
print('Edição:', edicao)
arquivo_xhtml = raspa_pdf(edicao, edicao + '.xhtml', destino_diretorio = '.')
print('XHTML:', arquivo_xhtml)
arquivo_spacy = analisa_txt(arquivo_xhtml, arquivo_xhtml + '.spacy', origem_diretorio = '.', destino_diretorio = '.')
print('Spacy:', arquivo_spacy)

In [3]:


dos_list = os.listdir(DIRETORIO_PAR)
dos_len = len(dos_list)

do_current = 0

do_queue = queue.Queue()
for do in dos_list:
    do_queue.put(do)

n_threads = 2

thread_queue = queue.Queue(maxsize = n_threads)

def producer():
    while not do_queue.empty():
        if not thread_queue.full():
            thread_queue.put(do_queue.get())
            do_queue.task_done()
        else:
            time.sleep(0.1)
            
    return

def consumer(name):
    local = threading.local()
    local.name = name
    local.nlp = spacy.load('pt_core_news_lg', exclude = nlp_exclude_list)
    while not do_queue.empty() or not thread_queue.empty():
        if not thread_queue.empty():
            do = thread_queue.get()
            local.time_in = time.time()
            analisa_paragrafos(do, do + '.json_spacy', nlp = local.nlp)
            local.time_out = time.time()
            local.now = time.strftime('%H:%M:%S')
            print(f'{local.name} {local.now} ({(local.time_out - local.time_in):4.5f}): {do}')
            thread_queue.task_done()
        else:
            time.sleep(0.1)
    return
            
producer_thread = threading.Thread(target = producer, name = 'produtor')
producer_thread.start()

print('sleeping for 2 seconds')
time.sleep(2)

consumer_threads = []
consumer_nlps = []
for c in range(n_threads):
    consumer_thread = threading.Thread(target = consumer, args = (str(c)), name = 'consumidor' + str(c))
    consumer_threads.append(consumer_thread)
    
for c in consumer_threads:
    c.start()
    
for c in consumer_threads:
    c.join()

print('done??')
        
    
# for do in dos_list:
#     thread = threading.Thread(target = analisa_paragrafos, args = (do, do + '.json_spacy'))
#     thread_queue.put(thread)

# while not thread_queue.empty():
#     thread_queue.get().start()
#     time.sleep(1)
    

    
#for txt in os.listdir(DIRETORIO_PAR):
#    analisa_paragrafos(txt, txt + '.json_spacy')
#     arquivo_spacy = analisa_txt(txt, txt + '.spacy')

sleeping for 2 seconds
1 23:49:06 (33.13385): do_20180116_2923.pdf.xhtml.json
0 23:49:27 (54.12230): do_20160324_2660.pdf.xhtml.json
1 23:49:54 (48.72551): do_20170701_2846.pdf.xhtml.json
0 23:50:18 (51.47186): do_20200204_3221.pdf.xhtml.json
1 23:50:43 (48.91508): do_20200328_3243.pdf.xhtml.json
1 23:51:23 (40.05919): do_20190117_3067.pdf.xhtml.json
1 23:51:51 (28.01811): do_20170321_2806.pdf.xhtml.json
0 23:54:04 (226.41411): do_20171216_2912.pdf.xhtml.json
1 23:54:18 (146.78591): do_20200704_3280.pdf.xhtml.json
1 23:54:49 (31.20062): do_20170525_2831.pdf.xhtml.json
0 23:55:08 (63.65162): do_20181229_3060.pdf.xhtml.json
0 23:55:18 (9.40376): do_20170207_2789.pdf.xhtml.json
1 23:55:21 (31.39078): do_20160514_2680.pdf.xhtml.json
1 23:55:59 (38.33147): do_20160409_2666.pdf.xhtml.json
0 23:56:31 (73.41975): do_20190202_3074.pdf.xhtml.json
0 23:57:36 (65.55624): do_20170826_2869.pdf.xhtml.json
0 23:58:12 (35.13557): do_20200123_3216.pdf.xhtml.json
1 23:58:25 (146.25724): do_20181027_3037.

1 00:52:14 (69.77147): do_20171228_2916.pdf.xhtml.json
0 00:53:28 (89.14828): do_20171028_2894.pdf.xhtml.json
1 00:53:43 (88.96497): do_20211230_3496.pdf.xhtml.json
0 00:54:02 (33.87791): do_20171007_2886.pdf.xhtml.json
1 00:55:32 (108.88873): do_20170930_2883.pdf.xhtml.json
0 00:55:58 (115.73391): do_20220120_3504.pdf.xhtml.json
1 00:56:12 (40.51761): do_20180421_2962.pdf.xhtml.json
0 00:57:10 (72.37753): do_20180424_2963.pdf.xhtml.json
1 00:57:52 (99.63948): do_20170916_2877.pdf.xhtml.json
0 00:58:00 (49.69517): do_20190725_3144.pdf.xhtml.json
1 00:58:27 (35.35005): do_20180313_2946.pdf.xhtml.json
1 00:58:56 (29.01914): do_20160719_2707.pdf.xhtml.json
0 00:59:10 (69.88142): do_20201217_3346.pdf.xhtml.json
0 00:59:46 (36.58044): do_20190427_3108.pdf.xhtml.json
1 01:00:01 (64.96512): do_20191130_3195.pdf.xhtml.json
1 01:00:25 (23.44129): do_20171205_2907.pdf.xhtml.json
0 01:00:30 (43.89708): do_20160521_2683.pdf.xhtml.json
1 01:00:58 (33.26335): do_20160128_2637.pdf.xhtml.json
1 01:01:

0 01:48:16 (35.54211): do_20210406_3390.pdf.xhtml.json
1 01:48:35 (50.05221): do_20211216_3491.pdf.xhtml.json
1 01:49:01 (25.52594): do_20180301_2941.pdf.xhtml.json
1 01:49:20 (18.95839): do_20210323_3385.pdf.xhtml.json
1 01:49:47 (27.19065): do_20210220_3372.pdf.xhtml.json
0 01:49:54 (97.68732): do_20210930_3462.pdf.xhtml.json
0 01:50:11 (17.26340): do_20200604_3268.pdf.xhtml.json
0 01:50:35 (23.78704): do_20210518_3407.pdf.xhtml.json
1 01:50:48 (61.16115): do_20180118_2924.pdf.xhtml.json
0 01:51:10 (35.18139): do_20190323_3094.pdf.xhtml.json
0 01:51:38 (27.63306): do_20180120_2925.pdf.xhtml.json
1 01:52:02 (73.85554): do_20201105_3329.pdf.xhtml.json
1 01:52:30 (28.40414): do_20210209_3367.pdf.xhtml.json
0 01:52:42 (64.46563): do_20160223_2647.pdf.xhtml.json
0 01:53:10 (27.78087): do_20191126_3193.pdf.xhtml.json
0 01:53:46 (35.70157): do_20201112_3332.pdf.xhtml.json
1 01:54:01 (90.84124): do_20181122_3045.pdf.xhtml.json
1 01:55:04 (62.42096): do_20210911_3454.pdf.xhtml.json
0 01:55:15

0 02:47:18 (75.84194): do_20210304_3377.pdf.xhtml.json
0 02:48:05 (47.22414): do_20161105_2750.pdf.xhtml.json
1 02:48:49 (117.17196): do_20181011_3031.pdf.xhtml.json
0 02:49:25 (79.24132): do_20200222_3229.pdf.xhtml.json
1 02:49:26 (37.04383): do_20170124_2783.pdf.xhtml.json
0 02:50:19 (54.55268): do_20160329_2661.pdf.xhtml.json
0 02:50:33 (14.33803): do_20170606_2836.pdf.xhtml.json
1 02:50:44 (78.23878): do_20160430_2674.pdf.xhtml.json
1 02:50:58 (13.51169): do_20200714_3284.pdf.xhtml.json
0 02:51:32 (58.66550): do_20200926_3315.pdf.xhtml.json
1 02:51:42 (44.03050): do_20160820_2720.pdf.xhtml.json
0 02:51:57 (25.34131): do_20200519_3262.pdf.xhtml.json
1 02:52:13 (30.86293): do_20190713_3139.pdf.xhtml.json
0 02:52:22 (24.22406): do_20210610_3416.pdf.xhtml.json
0 02:52:40 (18.18852): do_20180306_2943.pdf.xhtml.json
1 02:53:18 (65.72400): do_20201114_3333.pdf.xhtml.json
0 02:53:34 (53.75046): do_20201119_3335.pdf.xhtml.json
1 02:53:43 (24.12993): do_20190302_3086.pdf.xhtml.json
0 02:53:5

0 03:45:34 (62.71988): do_20170822_2867.pdf.xhtml.json
1 03:46:11 (41.37060): do_20161229_2772.pdf.xhtml.json
0 03:46:15 (41.30814): do_20170907_2874.pdf.xhtml.json
0 03:46:40 (24.82807): do_20170815_2864.pdf.xhtml.json
1 03:46:48 (37.12442): do_20191102_3185.pdf.xhtml.json
0 03:47:01 (20.95982): do_20180522_2974.pdf.xhtml.json
1 03:47:35 (47.41632): do_20190730_3146.pdf.xhtml.json
1 03:47:59 (24.00591): do_20210608_3415.pdf.xhtml.json
0 03:48:23 (82.44593): do_20160331_2662.pdf.xhtml.json
1 03:48:35 (35.73950): do_20171207_2908.pdf.xhtml.json
1 03:49:16 (41.59910): do_20191123_3192.pdf.xhtml.json
1 03:50:10 (53.64200): do_20181018_3033.pdf.xhtml.json
0 03:50:37 (133.77509): do_20200104_3208.pdf.xhtml.json
0 03:51:32 (54.44898): do_20190430_3109.pdf.xhtml.json
0 03:52:14 (42.82968): do_20170420_2818.pdf.xhtml.json
0 03:52:35 (20.74976): do_20210720_3432.pdf.xhtml.json
0 03:52:48 (12.91904): do_20200407_3247.pdf.xhtml.json
0 03:53:41 (53.03536): do_20220517_3551.pdf.xhtml.json
0 03:54:0

1 04:50:32 (38.18535): do_20220430_3544.pdf.xhtml.json
0 04:50:40 (36.34202): do_20170318_2805.pdf.xhtml.json
1 04:51:04 (32.42248): do_20160607_2689.pdf.xhtml.json
0 04:51:14 (34.06335): do_20181220_3057.pdf.xhtml.json
1 04:52:58 (113.80449): do_20210807_3440.pdf.xhtml.json
1 04:53:19 (21.38223): do_20201208_3342.pdf.xhtml.json
0 04:53:40 (145.32157): do_20170905_2873.pdf.xhtml.json
1 04:54:21 (61.63518): do_20210925_3460.pdf.xhtml.json
0 04:54:38 (58.71114): do_20181004_3028.pdf.xhtml.json
0 04:56:01 (82.71849): do_20190112_3065.pdf.xhtml.json
0 04:56:59 (58.20292): do_20200917_3311.pdf.xhtml.json
0 04:57:26 (26.69669): do_20200507_3257.pdf.xhtml.json
0 04:58:14 (47.76195): do_20170527_2832.pdf.xhtml.json
0 04:59:15 (60.96166): do_20181020_3034.pdf.xhtml.json
0 04:59:51 (36.01089): do_20220426_3542.pdf.xhtml.json
0 05:00:24 (33.13331): do_20170114_2779.pdf.xhtml.json
0 05:01:00 (35.70611): do_20161126_2758.pdf.xhtml.json
0 05:01:47 (46.85851): do_20170819_2866.pdf.xhtml.json
0 05:02:

1 05:56:41 (42.51055): do_20171202_2906.pdf.xhtml.json
0 05:57:10 (45.91197): do_20191026_3183.pdf.xhtml.json
1 05:57:39 (58.57200): do_20171118_2901.pdf.xhtml.json
0 05:57:59 (48.97974): do_20170722_2855.pdf.xhtml.json
1 05:58:05 (25.72825): do_20170704_2847.pdf.xhtml.json
1 05:58:32 (26.84232): do_20161217_2767.pdf.xhtml.json
0 05:58:58 (59.13094): do_20160924_2734.pdf.xhtml.json
1 05:59:02 (29.90640): do_20210309_3379.pdf.xhtml.json
0 05:59:19 (21.08301): do_20220303_3521.pdf.xhtml.json
1 05:59:30 (27.73098): do_20180823_3011.pdf.xhtml.json
0 05:59:48 (29.57163): do_20210325_3386.pdf.xhtml.json
0 06:00:13 (24.31604): do_20171017_2889.pdf.xhtml.json
1 06:00:24 (54.55433): do_20171114_2900.pdf.xhtml.json
0 06:00:29 (16.07516): do_20200707_3281.pdf.xhtml.json
1 06:00:49 (25.33269): do_20210218_3371.pdf.xhtml.json
0 06:00:56 (27.68725): do_20200128_3218.pdf.xhtml.json
0 06:01:27 (30.82057): do_20210727_3435.pdf.xhtml.json
1 06:02:22 (92.93797): do_20180426_2964.pdf.xhtml.json
0 06:02:27

In [5]:
threading.enumerate()

[<_MainThread(MainThread, started 140535272654656)>,
 <Thread(Thread-2, started daemon 140535218087680)>,
 <Heartbeat(Thread-3, started daemon 140535209694976)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 140535184254720)>,
 <ParentPollerUnix(Thread-1, started daemon 140535175599872)>]

In [4]:
import matplotlib.pyplot as plt

statistics = {}
for token in doc:
    if not token.pos_ in statistics:
        statistics[token.pos_] = 0
    statistics[token.pos_] += 1

plt.bar(statistics.keys(), statistics.values(), width = 0.7)

NameError: name 'doc' is not defined